# 1. IMPORTING LIBRARIES, FUNCTION AND DEFINING GLOBAL VARIABLES

## 1.1 Importing Libraries

In [1]:
# Library needed to load the pdf
import fitz

# Library needed to process the text using regular expressions
import re

# Library needed to display or process the data in forms of dataframes
import pandas as pd

# Library needed to handle the operations in deep learning
import torch

# Library needed to convert the data into arrays for faster processing
import numpy as np

# Library to handle operating system related operations
import os

## 1.2 Importing Functions

In [2]:
# (OPTIONAL) Function to beautify the waiting process with a loading bar
from tqdm import tqdm as tqdm

# Function to process the text in English
from spacy.lang.en import English

# Function to convert paragraphs to sentences
from sentence_transformers import SentenceTransformer

# Function to provide utility services to process the text such as tokenization, sentencizer
from sentence_transformers import util

# Functions for loading the LLM model
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import AutoModelForSeq2SeqLM

# Function for fetching the paths to pdfs
from glob import glob

## 1.3 Defining Global Variables

In [3]:
# Global variable consisting of all the stop words
from spacy.lang.en import STOP_WORDS

# Global variable telling about the number of sentences in each chunk stored in the dictionary
SENTENCE_CHUNKS = 10

# Global variable storing the name of the model that is used for the embedding
EMBEDDING_MODEL = 'all-MiniLM-L12-v2'

# Global variable storing the names of the pdfs that are to be loaded to be fed into the RAG model
PDF_PATHS = list()

# Global variable storing the integer telling to fetch the top k similar records for further processing
K = 10

# Global variable storing the name of the LLM model that will be used for augmenting the similar data
LLM_MODEL = 'google/gemma-2b-it'

# (FOR TESTING) Global variable storing the query that user wants to ask
# QUERY = "What are some good practices in machine learning?"
# QUERY = "What is formula for sigmoid function? The value for s in the function is 5. Can you compute the answer for the sigmoid function."
QUERY = "What is Tanh activation function."

# Setting up the device agnostic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Global variable for setting the temperature of the LLM model i.e how much data should LLM generate
TEMPERATURE = 0.9

## Global variable defining the length of tokens that the LLM has to generate
MAX_NEW_TOKENS = 1024

# 2. DATA ACQUISITION

## 2.1 Getting the paths to all the pdfs in the `Dataset` folder

In [ ]:
PDF_PATHS = glob('.\\Dataset\\*.pdf')

In [ ]:
for idx, path in tqdm(enumerate(PDF_PATHS), total=len(PDF_PATHS)):
    print(f"{idx+1}. {path[10:-4]}")

## 2.2 Opening all the documents

In [ ]:
documents = list()
for path in tqdm(PDF_PATHS, total=len(PDF_PATHS)):
    doc = fitz.open(path)
    documents.append(doc)

In [ ]:
for doc in tqdm(documents, total=len(documents)):
    print(doc)

## 2.3 Getting the text from all the documents

In [ ]:
pages = dict()
for doc in tqdm(documents, total=len(documents)):
    for page_number, page in enumerate(doc):
        if(page_number<15): 
            continue
        page_number = len(pages)
        pages[page_number] = page.get_text()

In [ ]:
for page_number, page in tqdm(pages.items(), total=len(documents)):
    print(f"{page_number}. {pages[page_number]}")
    print()

## 2.4 Getting the metadata of each page

In [ ]:
pages_and_metadata = list()
for page_number, page in tqdm(pages.items(), total=len(pages)):
    metadata = dict()
    metadata['page_number'] = page_number
    metadata['raw_text'] = page
    metadata['number_of_characters'] = len(page)
    metadata['number_of_tokens'] = len(page)/4
    metadata['number_of_words'] = len(page.split())
    pages_and_metadata.append(metadata)

In [ ]:
df = pd.DataFrame(pages_and_metadata)

In [ ]:
df.describe().round(2)

In [ ]:
print(f"The number of tokens this model is being trained on are: {df["number_of_tokens"].sum()/1000000:.2f} million tokens")

In [ ]:
print(f"The number of pages in the database are: {len(pages_and_metadata)}")

## 2.4 Preprocessing the `raw_text` from metadata

In [ ]:
def convert_to_lowercase(text):
    new_text = text.lower()
    return new_text

In [ ]:
def remove_stopwords(text):
    new_text = []
    for word in text.split():
        if word not in STOP_WORDS:
            new_text.append(word)
    return " ".join(new_text)

In [ ]:
def remove_html_tags(text):
    new_text = re.sub(r"<!--.*?-->", "", text)
    return new_text

In [ ]:
def remove_newlines(text):
    new_text = re.sub(r"\n+", " ", text)
    return new_text

In [ ]:
def remove_multiple_spaces(text):
    new_text = text.replace("  ", " ")
    return new_text

In [ ]:
def remove_comments(text):
    new_text = re.sub(r"<!--.*?-->", "", text)
    return new_text

In [ ]:
def remove_unnecessary_text(text):
    new_text = text.replace("answer:","").replace("question", "").replace(":","").replace("  "," ")
    return new_text

In [ ]:
def preprocess_text(text):
    text = convert_to_lowercase(text)
    text = remove_stopwords(text)
    text = remove_html_tags(text)
    text = remove_newlines(text)
    text = remove_multiple_spaces(text)
    text = remove_comments(text)
    text = remove_unnecessary_text(text)
    return text

In [ ]:
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    page["formatted_text"] = preprocess_text(page["raw_text"])

In [ ]:
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    print(page["formatted_text"])
    print()

## 2.5 Converting the paragraphs to sentences

In [ ]:
nlp = English()
nlp.add_pipe('sentencizer')
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    sentences = nlp(page["formatted_text"]).sents
    sentences = list(set([str(sentence).strip() for sentence in sentences if len(str(sentence).split())>10]))
    pages_and_metadata[page["page_number"]]["sentences"] = sentences

In [ ]:
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    print(page["sentences"])
    print()

## 2.6 Update the metadata

In [ ]:
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    pages_and_metadata[page["page_number"]]["number_of_sentences"] = len(page["sentences"])

In [ ]:
for key in pages_and_metadata[0].keys():
    print(key)

## 2.7 Converting sentences to sentence_chunks

In [ ]:
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    sentences = pages_and_metadata[page["page_number"]]["sentences"]
    sentence_chunk = [sentences[i : i+SENTENCE_CHUNKS] for i in range(0, len(sentences), SENTENCE_CHUNKS)]
    page["sentence_chunk"] = sentence_chunk

In [ ]:
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    print(page["sentence_chunk"])
    print()

## 2.8 Converting sentences into sentence embeddings

In [ ]:
embedding_model = SentenceTransformer(model_name_or_path=EMBEDDING_MODEL).to(device)

In [ ]:
for page in tqdm(pages_and_metadata):
    embeddings = list()
    # print(f"Processing Page {page['page_number']}")
    for sentence in page["sentences"]:
        # sentence = sentence.to(device)
        embedding = embedding_model.encode(sentence, batch_size=1024, convert_to_tensor=True, show_progress_bar=False, device=device)
        embedding = np.stack(embedding.tolist(), axis=0)
        embedding = torch.tensor(embedding)
        embedding = embedding.type(torch.float32)
        embeddings.append(embedding)
    sentence_embeddings = [np.array(embedding) for embedding in embeddings]
    pages_and_metadata[page["page_number"]]["embeddings"] = sentence_embeddings

In [ ]:
for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    print(np.array(page["embeddings"]).shape)

## 2.9 Checking the metadata present for use

In [ ]:
for key in pages_and_metadata[0].keys():
    print(key)

# 3. FETCHING SIMILAR CONTENT

## 3.1 Getting the data embeddings

In [ ]:
pages_and_metadata_embeddings = []

for page in tqdm(pages_and_metadata, total=len(pages_and_metadata)):
    page_embeddings = []
    for chunk_embedding in pages_and_metadata[page["page_number"]]["embeddings"]:
        if isinstance(chunk_embedding, torch.Tensor):
            chunk_embedding = chunk_embedding.tolist()
        page_embeddings.append(chunk_embedding)
    pages_and_metadata_embeddings.append(page_embeddings)

## 3.2 Converting each embedding into the same dimensions

In [ ]:
if pages_and_metadata_embeddings:
    embedding_dim = len(pages_and_metadata_embeddings[0][0])
    pages_and_metadata_embeddings = [
            [np.pad(chunk, (0, max(0, embedding_dim - len(chunk))), mode='constant')[:embedding_dim]
             for chunk in page]
            for page in pages_and_metadata_embeddings
        ]

## 3.3 Flattening the nested list of embeddings and the sentence to fetch by index

In [ ]:
flat_embeddings = [chunk for page in pages_and_metadata_embeddings for chunk in page]
flat_data = [sentence for page in pages_and_metadata for sentence in page["sentences"]]

## 3.4 Saving the flattened embeddings and the flattened data

In [ ]:
df = pd.DataFrame(flat_embeddings)
df.to_csv("embeddings_v5.csv", index=False)

df = pd.DataFrame(flat_data)
df.to_csv("data_v5.csv", index=False)

## 3.5 Loading the flattened embeddings and flattened data

In [4]:
flat_embeddings = pd.read_csv("embeddings_v5.csv").to_numpy()
flat_data = pd.read_csv("data_v5.csv")["0"].tolist()

In [5]:
print(f"The size of the flat_embeddings is {len(flat_embeddings)}")
print(f"The size of the flat_data is {len(flat_data)}")

The size of the flat_embeddings is 92230
The size of the flat_data is 92230


## 3.6 Converting embeddings to numpy array

In [6]:
pages_and_metadata_embeddings = np.array(flat_embeddings, dtype=np.float32)

## 3.7 Converting the numpy array embeddings to torch tensors

In [7]:
pages_and_metadata_embeddings = torch.tensor(pages_and_metadata_embeddings, dtype=torch.float32).to(device)

## 3.8 Getting the similarity score by query

In [8]:
embedding_model = SentenceTransformer(EMBEDDING_MODEL)
query_embeddings = embedding_model.encode(QUERY, convert_to_tensor=True).to(device)
dot_score = util.dot_score(query_embeddings, pages_and_metadata_embeddings)[0]

## 3.9 Getting the top k similar scores

In [9]:
top_scores, top_indices = torch.topk(dot_score, k=K)

In [10]:
print(f"Top scores: {top_scores}")
print(f"Top indices: {top_indices}")

Top scores: tensor([0.7715, 0.7452, 0.6983, 0.6900, 0.6900, 0.6748, 0.6615, 0.6551, 0.6482,
        0.6480], device='cuda:0')
Top indices: tensor([10490, 83082, 87417, 43686, 31166,  4333, 84253, 87936, 10046, 18902],
       device='cuda:0')


## 3.10 Getting the top k content based on the scores

In [11]:
context = list()
for index in top_indices:
    print(f"Fetching data from page {index}")
    context.append(flat_data[index.item()])

Fetching data from page 10490
Fetching data from page 83082
Fetching data from page 87417
Fetching data from page 43686
Fetching data from page 31166
Fetching data from page 4333
Fetching data from page 84253
Fetching data from page 87936
Fetching data from page 10046
Fetching data from page 18902


In [12]:
print(context)

['problem • tanh activation function times earlier book i’ve mentioned arguments tanh function better activation function sigmoid function.', 'hyperbolic tangent (tanh) \ue000e hyperbolic tangent tanh.x/ d e2x', 'furthermore, existence symmetries particular property ‘tanh’ function applies wide range activation functions (k˙urkov´a kainen, 1994).', 'tanh similar identity function near , training deep neural 0 network ˆy = w\ue03etanh(u \ue03etanh(v \ue03ex)) resembles training linear model ˆy = w\ue03eu\ue03ev \ue03ex long activations network kept small.', 'tanh similar identity function near , training deep neural 0 network ˆy = w\ue03etanh(u \ue03etanh(v \ue03ex)) resembles training linear model ˆy = w\ue03eu\ue03ev \ue03ex long activations network kept small.', 'logistic function f(a) = 1 1 + exp(−a) (5.17) tanh activation function f(a) = tanh(a) = exp(a) −exp(−a) exp(a) + exp(−a) . (', 'update candidate z computed linear combination xj hj', '5.191) exists equivalent network, comput

# 4. Augmentation

## 4.1 Login to HuggingFace CLI

In [13]:
from huggingface_hub import notebook_login
notebook_login()

## 4.2 Loading the LLM model

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=False,
).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OSError: Insufficient system resources exist to complete the requested service. (os error 1450)

## 4.3 Augmenting the prompt for instructing the LLM in a better way

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
context = "\n -".join(context)
base_prompt = f'''Based on the following context items, please answer the query
Context Items:
{context}
Query:
{QUERY}
Answer:'''

In [ ]:
base_prompt = base_prompt.format(context=context, query=QUERY)

## 4.4 Creating the dialogue template for the LLM

In [ ]:
dialogue_template = [{
    "role": "user",
    "content": base_prompt,
}]

## 4.5 Applying the prompt to the dialogue template

In [ ]:
prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False,
                                       add_generation_prompt=True)

## 4.6 Providing the prompt and retrieving the answer from the LLM model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
input_ids = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**input_ids, temperature=TEMPERATURE, do_sample=True, max_new_tokens=MAX_NEW_TOKENS)
output_text = tokenizer.decode(outputs[0])

In [ ]:
print(output_text)

In [ ]:
idx = output_text.find("Answer")

In [ ]:
answer = output_text[idx+7:]

In [ ]:
answer = answer.replace("**", "")
answer = answer.replace("<start_of_turn>model","")
answer = re.sub("<.*?>", "", answer)
# answer = answer[]

In [ ]:
print(f"The cleaned answer is: {answer}")